In [ ]:
# Load data and split

from sklearn.model_selection import train_test_split
import scipy

X = scipy.sparse.load_npz("vectors/vectors_stemmed_tf_idf_title_header_recitals.npz")
y = scipy.sparse.load_npz("vectors/concepts_stemmed_tf_idf_title_header_recitals.npz")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)

In [ ]:
# XGBoost model
# Training stopped at 2.5h hour mark in local
# Training stopped at 1h hour mark on cloud
# Memory intensive, multiple trainings failed in the process
# No convergence, low verbosity

import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

# create XGBoost instance with default hyper-parameters
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic', verbosity=3)

# create MultiOutputClassifier instance with XGBoost model inside
multilabel_model = MultiOutputClassifier(xgb_estimator)

# fit the model
multilabel_model.fit(X_train.todense(), y_train.todense(), verbose=True)

# # evaluate on test data
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, multilabel_model.predict(X_test))*100))

In [ ]:
# KNN addaptation for multi-label problem
# No verbosity, tested in cloud, slow to train and use (~1h training)
#!pip install scikit-multilearn

from skmultilearn.adapt import MLkNN
from sklearn.metrics import classification_report

classifier = MLkNN(k=3)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

# Show report
import numpy
from sklearn.metrics import classification_report
y_pred = (predictions > 0.5)
print("Testing documents: " + str(X_test.shape[0]))
print("Concepts in test data: " + str(len(numpy.unique(numpy.ravel(numpy.argmax((y_test.todense() > 0.5) , axis=1))))))
y_pred_bool = numpy.argmax(predictions.todense(), axis=1)
print("Concepts predicted: " + str((numpy.unique(numpy.ravel(numpy.argmax((predictions.todense() > 0.5) , axis=1)))).shape[0]))

print(classification_report(y_test, predictions, zero_division=0)) # , labels=numpy.unique(y_pred_bool)

In [ ]:
# SVC alghoritm addapted to multi-label problem with MultiOutputClassifier
# Training stopped at 3h hour mark on cloud
# No convergence, no verbosity to have information about learning

from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

svc = SVC(gamma="scale", verbose=True)
model = MultiOutputClassifier(estimator=svc)
model.fit(X_train.todense(), y_train.todense())
print(model.score(X_train, y_train))